In [2]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import pandas as pd 
import numpy as np 

In [3]:
data_item = pd.read_pickle('/kaggle/input/new-data-fc/data_item.pkl') 
data_user = pd.read_pickle('/kaggle/input/new-data-fc/data_user.pkl')
data_rec = pd.read_pickle('/kaggle/input/new-data-fc/data_rec_fearure.pkl')

In [3]:
data_item.head()

,app_id,title,rating,positive_ratio,user_reviews,price_final,tags,features
0,13500,Prince of Persia: Warrior Within™,0.875,0.84,2.199,0.999,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.875, 0.84, 2.199, 0.999]"
1,22364,BRINK: Agents of Change,0.625,0.85,0.021,0.299,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[0.625, 0.85, 0.021, 0.29900000000000004]"
2,113020,Monaco: What's Yours Is Mine,0.875,0.92,3.722,1.499,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.875, 0.92, 3.722, 1.499]"
3,226560,Escape Dead Island,0.500,0.61,0.873,1.499,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.5, 0.61, 0.873, 1.499]"
4,249050,Dungeon of the ENDLESS™,0.875,0.88,8.784,1.199,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.875, 0.88, 8.784, 1.199]"


In [4]:
data_user.head()

,user_id,products,reviews,mean_price,tags,features
0,11316351,0.677,0.068,1.391882,"[31.0, 1.0, 5.0, 1.0, 4.0, 1.0, 3.0, 3.0, 1.0,...","[0.677, 0.068, 1.3918823529411763]"
1,4363012,1.001,0.072,1.530417,"[31.0, 9.0, 9.0, 9.0, 0.0, 1.0, 7.0, 1.0, 0.0,...","[1.001, 0.072, 1.5304166666666665]"
2,4893896,0.496,0.050,1.538980,"[13.0, 4.0, 13.0, 0.0, 2.0, 0.0, 1.0, 2.0, 0.0...","[0.496, 0.05, 1.53898]"
3,6366584,1.597,0.129,1.635047,"[51.0, 31.0, 17.0, 3.0, 3.0, 1.0, 1.0, 0.0, 0....","[1.597, 0.129, 1.635046511627907]"
4,11140739,0.286,0.079,2.056241,"[20.0, 25.0, 13.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0....","[0.286, 0.079, 2.056240506329114]"


In [5]:
data_rec.head()

,app_id,is_recommended,hours,user_id,tags_user,features_user,tags_item,features_item
0,1325200,1,0.0945,11593837,"[70.0, 79.0, 5.0, 19.0, 0.0, 4.0, 9.0, 5.0, 0....","[0.734, 0.218, 2.598825688073394]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.875, 0.88, 24.353, 4.9990000000000006]"
1,536270,0,0.0007,1663073,"[9.0, 4.0, 1.0, 3.0, 2.0, 0.0, 3.0, 6.0, 0.0, ...","[0.162, 0.048, 2.616166666666667]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.875, 0.8, 6.274, 3.999]"
2,1063660,1,0.0141,3274203,"[109.0, 34.0, 9.0, 17.0, 4.0, 1.0, 9.0, 2.0, 0...","[0.39, 0.234, 1.8898504273504269]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.95, 7.341, 2.9989999999999997]"
3,1325200,0,0.1472,8531123,"[27.0, 8.0, 5.0, 5.0, 1.0, 3.0, 5.0, 4.0, 0.0,...","[1.253, 0.07, 1.7217571428571423]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.875, 0.88, 24.353, 4.9990000000000006]"
4,1325200,1,0.0438,4334710,"[27.0, 5.0, 1.0, 2.0, 4.0, 0.0, 5.0, 2.0, 1.0,...","[0.316, 0.098, 2.0993367346938774]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.875, 0.88, 24.353, 4.9990000000000006]"


In [6]:
data_user['idx_user'] = [i for i in range(len(data_user))] 

In [7]:
data_item['idx_item'] = [i for i in range(len(data_item))]

In [8]:
data = pd.merge(data_rec,data_user[['idx_user','user_id']],on ='user_id')
data = pd.merge(data,data_item[['idx_item','app_id']], on = 'app_id')

In [9]:
data.head()

,app_id,is_recommended,hours,user_id,tags_user,features_user,tags_item,features_item,idx_user,idx_item
0,1325200,1,0.0945,11593837,"[70.0, 79.0, 5.0, 19.0, 0.0, 4.0, 9.0, 5.0, 0....","[0.734, 0.218, 2.598825688073394]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.875, 0.88, 24.353, 4.9990000000000006]",22839,6740
1,536270,0,0.0007,1663073,"[9.0, 4.0, 1.0, 3.0, 2.0, 0.0, 3.0, 6.0, 0.0, ...","[0.162, 0.048, 2.616166666666667]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.875, 0.8, 6.274, 3.999]",2766,45945
2,1063660,1,0.0141,3274203,"[109.0, 34.0, 9.0, 17.0, 4.0, 1.0, 9.0, 2.0, 0...","[0.39, 0.234, 1.8898504273504269]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.95, 7.341, 2.9989999999999997]",12634,46296
3,1325200,0,0.1472,8531123,"[27.0, 8.0, 5.0, 5.0, 1.0, 3.0, 5.0, 4.0, 0.0,...","[1.253, 0.07, 1.7217571428571423]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.875, 0.88, 24.353, 4.9990000000000006]",38947,6740
4,1325200,1,0.0438,4334710,"[27.0, 5.0, 1.0, 2.0, 4.0, 0.0, 5.0, 2.0, 1.0,...","[0.316, 0.098, 2.0993367346938774]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.875, 0.88, 24.353, 4.9990000000000006]",50657,6740


In [10]:
data_user.head()

,user_id,products,reviews,mean_price,tags,features,idx_user
0,11316351,0.677,0.068,1.391882,"[31.0, 1.0, 5.0, 1.0, 4.0, 1.0, 3.0, 3.0, 1.0,...","[0.677, 0.068, 1.3918823529411763]",0
1,4363012,1.001,0.072,1.530417,"[31.0, 9.0, 9.0, 9.0, 0.0, 1.0, 7.0, 1.0, 0.0,...","[1.001, 0.072, 1.5304166666666665]",1
2,4893896,0.496,0.050,1.538980,"[13.0, 4.0, 13.0, 0.0, 2.0, 0.0, 1.0, 2.0, 0.0...","[0.496, 0.05, 1.53898]",2
3,6366584,1.597,0.129,1.635047,"[51.0, 31.0, 17.0, 3.0, 3.0, 1.0, 1.0, 0.0, 0....","[1.597, 0.129, 1.635046511627907]",3
4,11140739,0.286,0.079,2.056241,"[20.0, 25.0, 13.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0....","[0.286, 0.079, 2.056240506329114]",4


In [11]:
del data['app_id']
del data['user_id']

In [12]:
data['hours'] = data['hours'].apply(lambda x: x*10000)

In [13]:
# Define rating intervals and corresponding ratings 
rating_intervals = [(0.0, 5.60),(5.60,17.00) ,(17.00, 44.20), (44.20, 136.20),(136.20,float('inf'))]  # Hours intervals
ratings = [2,2.5,3,3.5,4]  # Corresponding ratings 
# Function to assign ratings based on hours played 
def assign_rating(hours, is_recommended):  
    for i, (start, end) in enumerate(rating_intervals): 
        if start <= hours < end: 
            return ratings[i] + 2 * is_recommended - 1
    return ratings[-1] + 2* is_recommended - 1  # Assign highest rating for hours beyond the last interval

# Apply rating assignment function
data['rating'] = data.apply(lambda row: assign_rating(row['hours'], row['is_recommended']), axis=1)

In [14]:
# train with 80 percent
data_train = pd.DataFrame()
data_test = pd.DataFrame()
for key, val in data.groupby('idx_user'): 
  a = int(len(val)*0.8) 
  val['id_item'] = [key for i in range(len(val))] 
  data_train = pd.concat([data_train, val[:a]], axis=0)
  data_test = pd.concat([data_test,val[a:]], axis = 0)

In [15]:
data_train.head()

,is_recommended,hours,tags_user,features_user,tags_item,features_item,idx_user,idx_item,rating,id_item
20964,1,172.0,"[31.0, 1.0, 5.0, 1.0, 4.0, 1.0, 3.0, 3.0, 1.0,...","[0.677, 0.068, 1.3918823529411763]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.875, 0.94, 35.237, 0.749]",0,45637,5.0,0
101616,0,14.0,"[31.0, 1.0, 5.0, 1.0, 4.0, 1.0, 3.0, 3.0, 1.0,...","[0.677, 0.068, 1.3918823529411763]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.875, 0.81, 25.663, 1.9989999999999999]",0,45820,1.5,0
180211,1,47.0,"[31.0, 1.0, 5.0, 1.0, 4.0, 1.0, 3.0, 3.0, 1.0,...","[0.677, 0.068, 1.3918823529411763]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.75, 0.75, 7.101, 1.499]",0,15087,4.5,0
193306,1,69.0,"[31.0, 1.0, 5.0, 1.0, 4.0, 1.0, 3.0, 3.0, 1.0,...","[0.677, 0.068, 1.3918823529411763]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.875, 0.93, 3.515, 0.799]",0,15919,4.5,0
315027,1,253.0,"[31.0, 1.0, 5.0, 1.0, 4.0, 1.0, 3.0, 3.0, 1.0,...","[0.677, 0.068, 1.3918823529411763]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.5, 0.56, 26.102, 0.9890000000000001]",0,16226,5.0,0


In [16]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# data user
def transform(data):
  tensor_id_user  = torch.Tensor(np.stack(data['idx_user'].values))
  tensor_id_item  = torch.Tensor(np.stack(data['idx_item'].values))
  tensor_rec  = torch.Tensor(np.stack(data['is_recommended'].values))
  tensor_rating = torch.Tensor(np.stack(data['rating'].values)) 
  tensor_feature_item  = torch.Tensor(np.stack(data['features_item'].values))
  tensor_feature_user = torch.Tensor(np.stack(data['features_user'].values))
  tensor_tag_item = torch.Tensor(np.stack(data['tags_item'].values))
  tensor_tag_user = torch.Tensor(np.stack(data['tags_user'].values))

  dataset = TensorDataset(tensor_id_user, tensor_id_item, tensor_rec, tensor_rating, tensor_feature_item, tensor_feature_user,tensor_tag_item,tensor_tag_user)
  dataloader = DataLoader(dataset, batch_size=128, shuffle=True)
  return dataloader


In [17]:
dataloader_train , dataloader_test = transform(data_train), transform(data_test)

In [18]:
for batch in dataloader_train:
    print(batch)
    if True:
        break

[tensor([16061.,  5801., 50942.,  4934., 43316.,  5057., 57448., 57201., 44240.,
        14606., 38217., 17011., 38960., 11887., 44283., 30031., 42862., 55767.,
         9593., 44508., 55390., 26480.,  7864., 29934., 28374., 48933., 28836.,
         4327.,  6934., 24065.,  6044., 18120., 46828., 25396.,  1502.,  4470.,
         3556., 46119., 25282., 20786., 10593., 43038., 17123., 31058., 54982.,
        57406., 15580., 53039.,  4461., 54530., 15074., 27205., 34457., 41857.,
        27344., 10181., 49857., 29582.,  4658., 24812., 44450., 43693., 54726.,
        37086., 28906., 51783., 31505., 47460.,  8204., 47472., 29492., 25038.,
        34410., 32430.,   818., 48481., 29390., 19081., 38747.,  8423.,  5972.,
         4205., 29918., 28639.,  1599., 44159., 24932., 22542., 45189., 43358.,
        39877., 57752., 15672., 25487., 51314., 32514., 55153., 32909., 54954.,
        28324., 17079., 27874., 46081., 53264.,  1771., 27637., 34597., 37441.,
        21268.,  7265., 37566., 30260.,

In [6]:
class NCF_Feature(nn.Module):
    def __init__(self,num_tag,dim_user,dim_item):
        super(NCF_Feature,self).__init__()
        self.linear_user = nn.Linear(in_features = dim_user, out_features = 10)
        self.linear_item = nn.Linear(in_features = dim_item, out_features = 10)
        
        self.embedding_tag = nn.Embedding(num_embeddings=num_tag,embedding_dim = 5)

        self.bn1 = nn.BatchNorm1d(10)
        self.bn2 = nn.BatchNorm1d(10)

        self.dense1 = nn.Linear(in_features = 30, out_features = 64)
        self.dense2 = nn.Linear(in_features = 64, out_features = 32)
        self.dense3 = nn.Linear(in_features = 32, out_features = 1)
        self.relu = nn.ReLU()

    def forward(self,user_idx,item_idx,user_feature, item_feature,user_tag,item_tag):
        user = self.linear_user(user_feature)
        item = self.linear_item(item_feature)
        
        user = self.bn1(user)
        item = self.bn2(item)

        # embedding tag 
        embedd_user_tag = torch.zeros(len(user_tag),5)
        for k in range(len(user_tag)):
            nonzero_indices = (user_tag[k] != 0).nonzero(as_tuple=True)[0]
            embeddings = self.embedding_tag(nonzero_indices)
            weighted_embeddings = embeddings * (user_tag[k][nonzero_indices].unsqueeze(1)/100)
            embedd_user_tag[k] = torch.sum(weighted_embeddings,dim = 0)

        embedd_item_tag = torch.zeros(len(item_tag),5)
        for k in range(len(item_tag)):
            one_hot_index = (item_tag[k] == 1).nonzero(as_tuple=True)[0]
            embedd_item_tag[k] = self.embedding_tag(one_hot_index)
        new_user = torch.concat((user,embedd_user_tag),dim = 1) 
        new_item = torch.concat((item,embedd_item_tag), dim = 1)  
        
        feature_concat = torch.concat((new_user,new_item),dim = 1)
        
        x = self.dense1(feature_concat)
        x = self.relu(x)
        x = self.dense2(x)
        x = self.relu(x)
        x = self.dense3(x)
        x = self.relu(x)
        return x 

In [9]:
num_user = len(data_user)
num_item = len(data_item)
num_tag = len(data_item.loc[0]['tags'])

In [10]:
dim_user = 3
dim_item = 4
model = NCF_Feature(num_tag,dim_user,dim_item)

In [21]:
print(model)

NCF_Feature(
  (linear_user): Linear(in_features=3, out_features=10, bias=True)
  (linear_item): Linear(in_features=4, out_features=10, bias=True)
  (embedding_tag): Embedding(10, 5)
  (bn1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense1): Linear(in_features=30, out_features=64, bias=True)
  (dense2): Linear(in_features=64, out_features=32, bias=True)
  (dense3): Linear(in_features=32, out_features=1, bias=True)
  (relu): ReLU()
)


In [22]:
learning_rate = 0.002
epoch = 15
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr= learning_rate)



In [24]:
device = ('cuda' if torch.cuda.is_available()
            else 'mps' if torch.backends.mps.is_available() else 'cpu')

In [25]:
from sklearn.metrics import ndcg_score, f1_score

# evaluation metric
user = data_user['idx_user'].unique()
# ndcg, f1
def metric(model):
    a = np.random.randint(0,len(user)-500)
    user_eval = user[a:a+500]
    ndcg = 0 
    f1 = 0
    for u in user_eval:
        item = list(data[data['idx_user'] == u]['idx_item'])
        actual = list(data[data['idx_user'] == u]['is_recommended'])
        s = predict(model,u,item)

        ndcg += ndcg_score([actual],[s])
        f1 += f1_score(np.array(actual),(np.array(s) >= 0.9).astype(int))
    result_ndcg = ndcg/len(user_eval)
    result_f1 = f1/len(user_eval)
    return result_ndcg,result_f1
    
def predict(model,u_idx,item):
    score = []
    for i in item:
        predict = model(torch.Tensor([u_idx]).long(),torch.Tensor([i]).long())
        score.append(predict.item())
    return score

In [25]:
for i in range(epoch): 
    model.train() 
    total_loss_train = 0 
    for batch in dataloader_train:
        id_user,id_item,rec, rating, feature_item, feature_user,item_tag,user_tag = batch
        id_user,id_item,rating, feature_item, feature_user,user_tag,item_tag = id_user.to(device), id_item.to(device), rating.to(device), feature_item.to(device), feature_user.to(device),user_tag.to(device),item_tag.to(device)

        output = model(id_user.long(),id_item.long(),feature_user, feature_item,user_tag,item_tag) 
        loss = loss_function(output.reshape(-1),rating.float()) 
        total_loss_train += loss.item() 
        optimizer.zero_grad() 
        loss.backward() 
        optimizer.step() 

#     total_loss_test = 0 
#     for batch in dataloader_test: 
#       model.eval() 
#       id_user,id_item,rec, hours, feature_item, feature_user,item_tag, user_tag = batch 
#       predict = model(id_user.long(),id_item.long(),feature_user,feature_item,user_tag,item_tag) 
#       loss = loss_function(predict.reshape(-1),rec.float()) 
#       total_loss_test += loss.item()/len(batch) 
#     model.eval()
#     ndcg,f1 = metric(model)

    print(f'Epoch{i +1}/{epoch}, loss_train : {total_loss_train/len(dataloader_train)}')

Epoch1/15, loss_train : 1.0903915854245105
Epoch2/15, loss_train : 1.0552401174591333
Epoch3/15, loss_train : 1.048996003092826
Epoch4/15, loss_train : 1.0452180379193756
Epoch5/15, loss_train : 1.0416598344725276
Epoch6/15, loss_train : 1.0388577203932943
Epoch7/15, loss_train : 1.0368178762781892
Epoch8/15, loss_train : 1.0352582311257998
Epoch11/15, loss_train : 1.0326258751915816
Epoch12/15, loss_train : 1.0316931916447378
Epoch13/15, loss_train : 1.03148304942791
Epoch14/15, loss_train : 1.0306961125271459
Epoch15/15, loss_train : 1.0305907523742894


In [27]:
data_train.to_pickle('data_train.pkl')
data_test.to_pickle('data_test.pkl')

In [26]:
torch.save(model.state_dict(), 'model_NCF_Feature.pth')

# evaluation model ndcg and f1 in test set

In [12]:
model_2 = NCF_Feature(num_tag,dim_user,dim_item)
model_2.load_state_dict(torch.load('/kaggle/input/model-val/model_NCF_Feature.pth'))

<All keys matched successfully>

In [13]:
data_test = pd.read_pickle('/kaggle/input/data-eval/data_test.pkl')

In [5]:
data_test.head()

,is_recommended,hours,tags_user,features_user,tags_item,features_item,idx_user,idx_item,rating,id_item
2716491,1,32.0,"[31.0, 1.0, 5.0, 1.0, 4.0, 1.0, 3.0, 3.0, 1.0,...","[0.677, 0.068, 1.3918823529411763]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.75, 0.77, 19.406, 0.099]",0,8961,4.0,0
2779811,1,6.0,"[31.0, 1.0, 5.0, 1.0, 4.0, 1.0, 3.0, 3.0, 1.0,...","[0.677, 0.068, 1.3918823529411763]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.875, 0.93, 0.369, 1.499]",0,7731,3.5,0
2803422,1,2.0,"[31.0, 1.0, 5.0, 1.0, 4.0, 1.0, 3.0, 3.0, 1.0,...","[0.677, 0.068, 1.3918823529411763]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.98, 9.482, 0.0]",0,40137,3.0,0
2878771,1,70.0,"[31.0, 1.0, 5.0, 1.0, 4.0, 1.0, 3.0, 3.0, 1.0,...","[0.677, 0.068, 1.3918823529411763]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.875, 0.94, 61.816, 1.9989999999999999]",0,45723,4.5,0
2889529,0,30.0,"[31.0, 1.0, 5.0, 1.0, 4.0, 1.0, 3.0, 3.0, 1.0,...","[0.677, 0.068, 1.3918823529411763]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.5, 0.53, 0.997, 0.499]",0,26816,2.0,0


In [21]:
from sklearn.metrics import ndcg_score, f1_score
def metric_2(model,k = 10):
    user = list(data_test['idx_user'].unique())
    a = np.random.randint(0,len(user)-500)
    user_eval = user[a:a+500]
    ndcg = 0
    f1 = 0
    for u in user_eval:
        u_interac_item = data_test[data_test['idx_user'] == u]
        score = predict_2(model,u,u_interac_item)
        score_item = {u_interac_item.iloc[i]['idx_item']:score[i] for i in range(len(score))}
        score_item = dict(sorted(score_item.items(), key=lambda item: item[1]))
        new_item = list(score_item.keys())[:k]
        new_score = list(score_item.values())[:k]
        actual = list(data_test[(data_test['idx_item'].isin(new_item)) & (data_test['idx_user'] == u)]['is_recommended'])
        ndcg += ndcg_score([actual],[new_score])
        f1 += f1_score(np.array(actual),np.array([1 if new_score[i]>= 3.5 else 0 for i in range(len(new_score))]))
    result_ndcg = ndcg/len(user_eval)
    result_f1 = f1/len(user_eval)
    return result_ndcg,result_f1
    
def predict_2(model,u_idx,item):
    model.eval()
    score = []
    for i in range(len(item)):
        predict = model(torch.Tensor([u_idx]).long(),torch.Tensor([item.iloc[i]['idx_item']]).long(),torch.Tensor(item.iloc[i]['features_user']).unsqueeze(0), torch.Tensor(item.iloc[i]['features_item']).unsqueeze(0),torch.Tensor(item.iloc[i]['tags_user']).unsqueeze(0),torch.Tensor(item.iloc[i]['tags_item']).unsqueeze(0))
        score.append(predict.item())
    return score

In [24]:
for i in range(4):
    ndcg, f1 = metric_2(model_2, k = 10)
    print('evaluation' + str(i+1) + ' metric_ndcg: ' + str(ndcg) + 'metric_f1: ' + f1 )

TypeError: can only concatenate str (not "numpy.float64") to str

In [23]:
print(ndcg,f1)

0.8959034931409927 0.7633695891827484
